In [1]:
import os
import gc
import warnings 

import pandas as pd
from pandas.plotting import register_matplotlib_converters
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)
register_matplotlib_converters()
sns.set()

In [2]:
!pip install xgboost


In [3]:
data=pd.read_csv('data_filt_final.csv')
#data.drop('weights',inplace=True,axis=1)
data.head()

,Unnamed: 0,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,part,wm_yr_wk,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,moon_phase,shift_t42,shift_t49,shift_t60,shift_t63,shift_t28,shift_t56,shift_t28_rolling_std_t28,shift_t28_rolling_mean_t28,rolling_min_t28,rolling_max_t28,rolling_sum_t28,shift_t28_rolling_std_t56,shift_t28_rolling_mean_t56,rolling_min_t56,rolling_max_t56,rolling_sum_t56,shift_t28_rolling_std_t84,shift_t28_rolling_mean_t84,rolling_min_t84,rolling_max_t84,rolling_sum_t84,rolling_skew_t28,rolling_kurt_t28,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t30,shift_t28_log,shift_t28_sqrt,shift_t56_log,shift_t56_sqrt,shift_t28_diff_t7,shift_t56_diff_t7,year,quarter,month,week,day,dayofweek,is_weekend,month_day
0,19747357,4216,421,1,0,6,1,1851,0,1,11604,0.0,0.0,0.0,0.0,0,0,0,7.98,4,0.0,1.0,0.0,0.0,0.0,0.0,0.416270,0.107143,0.0,2.0,3.0,0.345190,0.089286,0.0,2.0,5.0,0.302030,0.071429,0.0,2.0,6.0,4.126069,17.401083,0.0,0.000000,8.603189e-09,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,2016,1,2,8,22,0,0,-34
1,19747355,4196,419,1,0,6,1,1851,1,1,11604,0.0,0.0,0.0,0.0,0,0,0,2.50,4,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.443935,0.196429,0.0,2.0,11.0,0.492631,0.214286,0.0,2.0,18.0,0.000000,0.000000,0.0,0.131944,2.090814e-08,0.000000e+00,0.000000,0.000000,0.0,0.0,0.0,0.0,2016,1,2,8,22,0,0,-34
2,19747356,4206,420,1,0,6,1,1851,0,1,11604,0.0,0.0,0.0,0.0,0,0,0,9.98,4,1.0,0.0,0.0,1.0,0.0,0.0,0.741798,0.571429,0.0,3.0,16.0,0.872189,0.553571,0.0,3.0,31.0,0.766855,0.452381,0.0,3.0,38.0,1.495646,2.888336,0.0,0.000000,1.265594e-08,1.371161e-08,0.000000,0.000000,0.0,0.0,-1.0,-1.0,2016,1,2,8,22,0,0,-34
3,19747358,4226,422,1,0,6,1,1851,1,1,11604,0.0,0.0,0.0,0.0,0,0,0,6.98,4,2.0,0.0,0.0,2.0,3.0,0.0,0.860663,1.000000,0.0,3.0,28.0,1.341520,1.267857,0.0,7.0,71.0,1.504249,1.380952,0.0,7.0,116.0,0.375381,-0.666277,0.0,0.000000,1.899533e-08,0.000000e+00,1.386294,1.732051,0.0,0.0,2.0,-2.0,2016,1,2,8,22,0,0,-34
4,19747364,4286,428,1,0,6,1,1851,0,1,11604,0.0,0.0,0.0,0.0,0,0,0,2.50,4,0.0,0.0,0.0,3.0,0.0,0.0,0.722832,0.321429,0.0,3.0,9.0,0.724345,0.357143,0.0,3.0,20.0,0.682542,0.333333,0.0,3.0,28.0,2.583960,6.888949,0.0,0.000000,0.000000e+00,2.767076e-09,0.000000,0.000000,0.0,0.0,0.0,-3.0,2016,1,2,8,22,0,0,-34


In [4]:
def split_data(data):

    #Product details upto day 1913 is used as training dataset
    X_train = data[data['d']<=1913]
    y_train = X_train['demand']
    X_train.drop('demand',axis = 1,inplace = True)
    
    # Product details from day 1914 to 1941 is used as validation dataset
    X_val = data[(data['d']>=1914) & (data['d']<=1941)]
    y_val = X_val['demand']
    X_val.drop(['demand'],axis = 1,inplace = True)
    
    # Product details starting from day 1942 is used as testing dataset
    X_test = data[(data['d']>=1942)]
    X_test.drop(['demand'],axis = 1,inplace = True)
    
    #print("Train dataset",X_train.shape,y_train.shape)
    #print("Validation dataset",X_val.shape,y_val.shape)
    #print("Test dataset",X_test.shape)
    return(X_train,y_train,X_val,y_val,X_test)

In [5]:
xtrain,ytrain,xval,yval,xtest=split_data(data)


In [6]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import xgboost as xg


## Running Grid Search CV for Ridge Regression

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

ridge_reg = linear_model.Ridge()
ridge_grid = [{'alpha': [0.001,0.01,0.1,1],'solver': ['lsqr','auto']}]


grid = GridSearchCV(ridge_reg, ridge_grid, verbose = 3, scoring='neg_root_mean_squared_error')
grid.fit(xtrain, ytrain)
grid.best_params_ # this gives us the best hyperparameters

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END .........alpha=0.001, solver=lsqr;, score=-3.431 total time=   1.8s
[CV 2/5] END .........alpha=0.001, solver=lsqr;, score=-3.606 total time=   1.6s
[CV 3/5] END .........alpha=0.001, solver=lsqr;, score=-3.558 total time=   1.4s
[CV 4/5] END .........alpha=0.001, solver=lsqr;, score=-3.771 total time=   1.8s
[CV 5/5] END .........alpha=0.001, solver=lsqr;, score=-3.499 total time=   1.6s
[CV 1/5] END ........alpha=0.001, solver=auto;, score=-34.729 total time=   1.6s
[CV 2/5] END .........alpha=0.001, solver=auto;, score=-2.353 total time=   3.9s
[CV 3/5] END .........alpha=0.001, solver=auto;, score=-2.228 total time=   1.2s
[CV 4/5] END .........alpha=0.001, solver=auto;, score=-2.329 total time=   0.9s
[CV 5/5] END .........alpha=0.001, solver=auto;, score=-2.218 total time=   0.9s
[CV 1/5] END ..........alpha=0.01, solver=lsqr;, score=-3.431 total time=   0.9s
[CV 2/5] END ..........alpha=0.01, solver=lsqr;, 

{'alpha': 0.001, 'solver': 'lsqr'}

## Running Grid Search CV for XG Boost Regressor

In [13]:
import xgboost as xg
from xgboost import XGBRegressor

xg = XGBRegressor()

xg_grid = [{
    'max_depth':[2,4],
    'n_estimators':[50,100],
    'eta':[0.3,0.5]
}]

grid = GridSearchCV(xg, xg_grid, verbose = 3, scoring='neg_root_mean_squared_error')
grid.fit(xtrain, ytrain)
grid.best_params_ 

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5] END eta=0.3, max_depth=2, n_estimators=50;, score=-2.265 total time=  18.8s
[CV 2/5] END eta=0.3, max_depth=2, n_estimators=50;, score=-2.341 total time=  18.9s
[CV 3/5] END eta=0.3, max_depth=2, n_estimators=50;, score=-2.203 total time=  18.9s
[CV 4/5] END eta=0.3, max_depth=2, n_estimators=50;, score=-2.310 total time=  18.8s
[CV 5/5] END eta=0.3, max_depth=2, n_estimators=50;, score=-2.146 total time=  20.0s
[CV 1/5] END eta=0.3, max_depth=2, n_estimators=100;, score=-2.236 total time=  38.5s
[CV 2/5] END eta=0.3, max_depth=2, n_estimators=100;, score=-2.317 total time=  40.0s
[CV 3/5] END eta=0.3, max_depth=2, n_estimators=100;, score=-2.181 total time=  43.1s
[CV 4/5] END eta=0.3, max_depth=2, n_estimators=100;, score=-2.282 total time=  37.8s
[CV 5/5] END eta=0.3, max_depth=2, n_estimators=100;, score=-2.131 total time=  36.9s
[CV 1/5] END eta=0.3, max_depth=4, n_estimators=50;, score=-2.209 total time=  37.6s


{'eta': 0.3, 'max_depth': 4, 'n_estimators': 100}

In [15]:
from sklearn import neighbors

## Running Grid Search CV for KNN

In [ ]:
knn = neighbors.KNeighborsRegressor()

p_grid_knn = [{'n_neighbors': range(1,5,1),
 'weights': ['uniform', 'distance']}]

grid = GridSearchCV(knn, p_grid_knn, verbose = 3, scoring='neg_root_mean_squared_error')
grid.fit(xtrain, ytrain)
grid.best_params_ 